In [1]:
#HSV hue saturationn value
import cv2 
import numpy as np 
cap=cv2.VideoCapture('./dataset1/bottles.mp4')

def nothing(x):
    pass 
cv2.namedWindow("tracker")
cv2.createTrackbar('LH', 'tracker',0,255,nothing)
cv2.createTrackbar('LS', 'tracker',0,255,nothing)
cv2.createTrackbar('LV', 'tracker',0,255,nothing)

cv2.createTrackbar('UH', 'tracker',255,255,nothing)
cv2.createTrackbar('US', 'tracker',255,255,nothing)
cv2.createTrackbar('UV', 'tracker',255,255,nothing)


while True:
    ret,frame=cap.read()
    b,g,r=cv2.split(frame)

    b+=20 
    g+=10
    r+=20

    frame=cv2.merge((b,g,r ))

    if ret:
        
        l_h=cv2.getTrackbarPos("LH","tracker")
        l_s=cv2.getTrackbarPos("LS","tracker")
        l_V=cv2.getTrackbarPos("LV","tracker")
    
        u_h=cv2.getTrackbarPos("UH","tracker")
        u_s=cv2.getTrackbarPos("US","tracker")
        u_V=cv2.getTrackbarPos("UV","tracker")
    
        l_b=np.array([l_h,l_s,l_V])#lower bound
        u_b=np.array([u_h,u_s,u_V])#upper  bound
    
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask=cv2.inRange(hsv,l_b,u_b) 
        res=cv2.bitwise_and(frame,frame,mask=mask)
    
        cv2.imshow('frame',frame)
        cv2.imshow('mask',mask)
        cv2.imshow('res',res)

    if cv2.waitKey(80)& 0xFF ==27: 
        break
cap.release()
cv2.destroyAllWindows()
    



In [2]:
#HSV value check for extraction 
import cv2 
import numpy as np 
cap=cv2.VideoCapture('./dataset1/bottles.mp4')


while True:
    ret,frame=cap.read()

    if ret:
    
        l_b=np.array([58,93,0])#lower bound
        u_b=np.array([90,255,255])#upper  bound
    
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask=cv2.inRange(hsv,l_b,u_b) 
        res=cv2.bitwise_and(frame,frame,mask=mask)
    
        cv2.imshow('frame',frame)
        cv2.imshow('mask',mask)
        cv2.imshow('res',res)

    if cv2.waitKey(80)& 0xFF ==27: 
        break
cap.release()
cv2.destroyAllWindows()
    



In [4]:
#HSV hue saturationn value
import cv2 
import numpy as np 
import imutils
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
cap=cv2.VideoCapture('./dataset1/bottles.mp4')
#for output video
img=cap.read()
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out=cv2.VideoWriter("output.mp4",fourcc,40.0,(1000,1000))
length=[]
def bottle_finder(x):
    if x>=19 and x<=21:
        cv2.putText(frame,'2 Liter',(10,60),cv2.FONT_HERSHEY_COMPLEX,1.5,(255,255,255),2)
    elif x>=16 and x<=17:
        cv2.putText(frame,'1.5 Liter ',(10,60),cv2.FONT_HERSHEY_COMPLEX,1.5,(255,255,255),2)
    elif x>=12 and x<=13:
        cv2.putText(frame,'medium 24 fl oz',(10,60),cv2.FONT_HERSHEY_COMPLEX,1.5,(255,255,255),2)
    elif x>=10 and x<=11:
        cv2.putText(frame,'small 8.55 fl oz',(10,60),cv2.FONT_HERSHEY_COMPLEX,1.5,(255,255,255),2)
    else:
        #cv2.putText(frame,'wait..',(10,60),cv2.FONT_HERSHEY_COMPLEX,1.5,(255,255,255),2)
        pass
        
while True:
    ret,frame=cap.read()

    if ret:
        
        b,g,r=cv2.split(frame)

        b+=20 
        g+=10
        r+=20

        frame=cv2.merge((b,g,r ))

        
        l_b=np.array([0,0,0])#lower bound
        u_b=np.array([111,229,204])#upper  bound
    
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask=cv2.inRange(hsv,l_b,u_b) 
        #res=cv2.bitwise_and(frame,frame,mask=mask)

        
        cnts=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts=imutils.grab_contours(cnts)

        (cnts, _) = contours.sort_contours(cnts)

        # Remove contours which are not large enough
        cnts = [x for x in cnts if cv2.contourArea(x) > 1000]

        ref_object = cnts[0]
        box  = cv2.minAreaRect(ref_object)
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        box = perspective.order_points(box)
        (tl, tr, br, bl) = box
        dist_in_pixel = euclidean(tl, tr)
        dist_in_cm = 2
        pixel_per_cm = dist_in_pixel/dist_in_cm
        
        # Draw remaining contours
        for cnt in cnts:
            box = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            box = perspective.order_points(box)
            (tl, tr, br, bl) = box
            cv2.drawContours(frame, [box.astype("int")], -1, (0, 0, 255), 2)
            mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2),
                         tl[1] + int(abs(tr[1] - tl[1])/2))
            mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2),
                       tr[1] + int(abs(tr[1] - br[1])/2))
            wid = euclidean(tl, tr)/pixel_per_cm
            ht = euclidean(tr, br)/pixel_per_cm
            length.append([wid,ht])
            cv2.putText(frame, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
            cv2.putText(frame, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

            bottle_finder(ht)

        cv2.imshow('frame',frame)
        out.write(frame)
        cv2.imshow('mask',mask)
        #cv2.imshow('threash',threash)

    if cv2.waitKey(1)& 0xFF ==27: 
        break
cap.release()
out.release()
cv2.destroyAllWindows()

